1. Install and Import dependencies

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
!pip3 install transformers beautifulsoup4 pandas numpy requests

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

2. Model Instance

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Encode and Calculate Sentiment

In [6]:
tokens = tokenizer.encode('Tasted bland, I actually hated it', return_tensors='pt')

In [7]:
result = model(tokens)

In [8]:
result.logits

tensor([[ 2.1336,  2.1120,  0.5858, -1.6214, -2.5580]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits))+1

1

4. Scrape Reviews

In [10]:
r = requests.get('https://www.yelp.com/biz/mogouyan-hand-pulled-noodles-toronto?osq=Mogouyan+Hand-Pulled+Noodles')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

5. Load Reviews and Score

In [11]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [12]:
df['review'].iloc[0]

"This is one of my favourite spot if you're looking for fresh hand pulled noodles \xa0This location is fairly popular and busy, so we were lucky to have been to get a seat during lunch time as spots filled up quickly. Their menu offers a range of appetizers, soup noodles, stir fried noodles and more!We wanted to try their soup noodle and an appetizer, and wow, I was surprised by the size of the large bowl. We got the Braised Beef Noodles with one spice, and as someone who can eat some spicy dishes, level 2 wouldn't hurt - however still flavourful! There were many chunks by braised beef, and not just skin. The noodles were very bouncy, and you can definitely definitely taste the difference between fresh noodles VS frozen. You could also choose the noodle style to go with \xa0your bowl. We opted for flat, but perhaps will try the thin flat next time as it was on the thicker side, and I got full really fast from it. Definitely enjoyed this dish overall I'm hoping to try all their noodles,

In [13]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [14]:
sentiment_score(df['review'].iloc[1])

5

In [15]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [16]:
df

,review,sentiment
0,This is one of my favourite spot if you're loo...,4
1,"Our waiter was so nice, friendly, and helpful....",5
2,This place is always bumping and a cold Saturd...,3
3,I was at the Sheraton and this restaurant pick...,5
4,"I have been to 3 different locations, each tim...",5
5,I got the braised beef noodle after having a g...,2
6,I had a really satisfying meal here the other ...,5
7,First time in Toronto. Crew craved noodles so ...,5
8,"For the price and size of food, absolutely del...",4
9,I am a sucker for hand pulled noodles. And thi...,2
